<a href="https://colab.research.google.com/github/dibsondivya/ai-health-event/blob/main/training/Divya_Finetuning_Distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Source
[Fine Tuning](https://medium.com/nlplanet/fine-tuning-distilbert-on-senator-tweets-a6f2425ca50e)

## Data and Package Installation
Manually import data files

In [22]:
import pandas as pd
df = pd.read_csv('train_textcleaned.csv', index_col=0)
df_test = pd.read_csv('test_textcleaned.csv', index_col=0)

In [23]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Prepare datasets

In [24]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Put clean data in a dataset split into train and test sets
dataset = Dataset.from_pandas(df).train_test_split(train_size=0.8, seed=123)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', '__index_level_0__'],
        num_rows: 2540
    })
    test: Dataset({
        features: ['tweet', 'label', '__index_level_0__'],
        num_rows: 635
    })
})


In [25]:
testdataset = Dataset.from_pandas(df_test)
print(testdataset)

Dataset({
    features: ['tweet', 'label', '__index_level_0__'],
    num_rows: 1361
})


## Tokenize data

In [26]:
!pip install transformers
from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
# Tokenize and encode the dataset
def tokenize(batch):
    tokenized_batch = tokenizer(batch['tweet'], padding=True, truncation=True, max_length=128)
    return tokenized_batch

dataset_enc = dataset.map(tokenize, batched=True, num_proc=4)

# Set dataset format for PyTorch
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Check the output
print(dataset_enc["train"].column_names)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

['tweet', 'label', '__index_level_0__', 'input_ids', 'attention_mask']


## Prepare dataloaders

In [28]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

# Instantiate a data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create data loaders for to reshape data for PyTorch model
train_dataloader = DataLoader(
    dataset_enc["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_enc["test"], batch_size=8, collate_fn=data_collator
)

## Load model

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

# Set number of class labels based on dataset since 0,1,2,3
num_labels = 4
print(f"Number of labels: {num_labels}")

# Load model from checkpoint
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels=num_labels)

Number of labels: 4


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [30]:
from transformers import AdamW
from transformers import get_scheduler

# Model parameters
learning_rate = 5e-5
num_epochs = 5

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Further define learning rate scheduler
num_training_batches = len(train_dataloader)
num_training_steps = num_epochs * num_training_batches
lr_scheduler = get_scheduler(
    "linear",                   # linear decay
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


## Device settings

In [31]:
# Set the device automatically (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model to device
model.to(device)

cpu


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

## Train model

In [32]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Train the model with PyTorch training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1590 [00:00<?, ?it/s]

## Save model

In [33]:
# Save model to disk
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define root dir in Google Drive
root_dir = "/content/drive/MyDrive/colab_data"

model.save_pretrained(f"{root_dir}/models/distilbert-ai-health")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# Install git-lfs
import huggingface_hub
huggingface_hub.lfs.install_lfs_in_userspace()

In [35]:
# Log in to Huggingface CLI
!transformers-cli login

WARNING! `transformers-cli login` is deprecated and will be removed in v5. Please use `huggingface-cli login` instead.

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: dibsondivya
Password: 
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:587: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.8.
  FutureWarning,
Login successful
Your token: sokRGmXrlITJxgId

In [36]:
import os

# Configure git settings
!git config --global user.email "divya_shridar@mymail.sutd.edu.sg"
!git config --global user.name "dibsondivya"

# Push PT model to hub
model.push_to_hub(
    "distilbert-phmtweets-sutd",                            # model name
    language="en",                                            # language
    dataset_tags="sutdai/phm-tweets",                # HF dataset used for training
    library_name="pytorch",
    metrics=["accuracy", "f1"],                               
    tags=["text-classification", "transformers", "pytorch"],  # model tags
    finetuned_from="distilbert-base-uncased",                 # base model
    # commit_message="..."
    )

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='distilbert-phmtweets-sutd' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:676: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/dibsondivya/distilbert-phmtweets-sutd into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

To https://huggingface.co/dibsondivya/distilbert-phmtweets-sutd
   00acd72..8c998d8  main -> main



'https://huggingface.co/dibsondivya/distilbert-phmtweets-sutd/commit/8c998d84f89931bcd79ba1baa10fc2d3a4774e55'

In [43]:
tokenizer.push_to_hub("distilbert-phmtweets-sutd")

To https://huggingface.co/dibsondivya/distilbert-phmtweets-sutd
   8c998d8..c3ce908  main -> main



'https://huggingface.co/dibsondivya/distilbert-phmtweets-sutd/commit/c3ce90892e451c26967bcd3c04f21183a78f4f12'

## Validate model

In [38]:
from datasets import load_metric

# Load metric
metric = load_metric("accuracy", "f1")

# Iteratively evaluate the model and compute metrics
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Get model accuracy and F1 score
metric.compute()

{'accuracy': 0.8708661417322835}

## Prepare test set

In [39]:
testdataset_enc = testdataset.map(tokenize, batched=True, num_proc=4)

# Set dataset format for PyTorch
testdataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataloader = DataLoader(
    testdataset_enc, batch_size=8, collate_fn=data_collator
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

## Evaluate with test data

In [40]:
# Iteratively evaluate the model and compute metrics
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# Get model accuracy and F1 score
metric.compute()

{'accuracy': 0.8772961058045555}

## Save model

In [41]:
#from google.colab import files 

#torch.save(model.state_dict(), 'distilbert.pth') # saves model parameters

# download distilbert file
#files.download('distilbert.pth')

## to load
#the_model = TheModelClass(*args, **kwargs)
#the_model.load_state_dict(torch.load(PATH))
#the_model.eval()

In [42]:
#torch.save(model, 'distilbertmodel.pth')
#files.download('distilbertmodel.pth')

## to load
# torch.load(PATH)